In [3]:
import argparse
import os
# from resnet20 import ResNetCIFAR
from model import Model
import pandas as pd
import torch
import torch.optim as optim
!pip install thop
from thop import profile, clever_format
from torch.utils.data import DataLoader
from tqdm import tqdm

import utils
# from model import Model
import torch.nn as nn



class ConstantWithWarmup(torch.optim.lr_scheduler._LRScheduler):
    def __init__(
        self,
        optimizer,
        num_warmup_steps: int,
    ):
        self.num_warmup_steps = num_warmup_steps
        super().__init__(optimizer)

    def get_lr(self):
        if self._step_count <= self.num_warmup_steps:
            # warmup
            scale = 1.0 - (self.num_warmup_steps - self._step_count) / self.num_warmup_steps
            lr = [base_lr * scale for base_lr in self.base_lrs]
            self.last_lr = lr
        else:
            lr = self.base_lrs
        return lr




# train for one epoch to learn unique features
def train(net, data_loader, train_optimizer):
    net.train()
    total_loss, total_num, train_bar = 0.0, 0, tqdm(data_loader)
    for pos_1, pos_2, target in train_bar:
        pos_1, pos_2 = pos_1.cuda(non_blocking=True), pos_2.cuda(non_blocking=True)
        feature_1, out_1, pred_1= net(pos_1)
        feature_2, out_2, pred_2 = net(pos_2)
        # [2*B, D]
        out = torch.cat([out_1, out_2], dim=0)
        # [2*B, 2*B]
        sim_matrix = torch.exp(torch.mm(out, out.t().contiguous()) / temperature)
        mask = (torch.ones_like(sim_matrix) - torch.eye(2 * batch_size, device=sim_matrix.device)).bool()
        # [2*B, 2*B-1]
        sim_matrix = sim_matrix.masked_select(mask).view(2 * batch_size, -1)

        # compute loss
        pos_sim = torch.exp(torch.sum(out_1 * out_2, dim=-1) / temperature)
        # [2*B]
        pos_sim = torch.cat([pos_sim, pos_sim], dim=0)
        loss = (- torch.log(pos_sim / sim_matrix.sum(dim=-1))).mean()
        train_optimizer.zero_grad()
        loss.backward()
        train_optimizer.step()

        total_num += batch_size
        total_loss += loss.item() * batch_size
        train_bar.set_description('Train Epoch: [{}/{}] Loss: {:.4f}'.format(epoch, epochs, total_loss / total_num))
    
    return total_loss / total_num


# test for one epoch, use weighted knn to find the most similar images' label to assign the test image
def test(net, memory_data_loader, test_data_loader):
    net.eval()
    total_top1, total_top5, total_num, feature_bank = 0.0, 0.0, 0, []
    with torch.no_grad():
        # generate feature bank
        for data, _, target in tqdm(memory_data_loader, desc='Feature extracting'):
            feature, out, pred = net(data.cuda(non_blocking=True))
            feature_bank.append(feature)
        # [D, N]
        feature_bank = torch.cat(feature_bank, dim=0).t().contiguous()
        # [N]
        feature_labels = torch.tensor(memory_data_loader.dataset.targets, device=feature_bank.device)
        # loop test data to predict the label by weighted knn search
        test_bar = tqdm(test_data_loader)
        for data, _, target in test_bar:
            data, target = data.cuda(non_blocking=True), target.cuda(non_blocking=True)
            feature, out, pred = net(data)

            total_num += data.size(0)
            # compute cos similarity between each feature vector and feature bank ---> [B, N]
            sim_matrix = torch.mm(feature, feature_bank)
            # [B, K]
            sim_weight, sim_indices = sim_matrix.topk(k=k, dim=-1)
            # [B, K]
            sim_labels = torch.gather(feature_labels.expand(data.size(0), -1), dim=-1, index=sim_indices)
            sim_weight = (sim_weight / temperature).exp()

            # counts for each class
            one_hot_label = torch.zeros(data.size(0) * k, c, device=sim_labels.device)
            # [B*K, C]
            one_hot_label = one_hot_label.scatter(dim=-1, index=sim_labels.view(-1, 1), value=1.0)
            # weighted score ---> [B, C]
            pred_scores = torch.sum(one_hot_label.view(data.size(0), -1, c) * sim_weight.unsqueeze(dim=-1), dim=1)

            pred_labels = pred_scores.argsort(dim=-1, descending=True)
            total_top1 += torch.sum((pred_labels[:, :1] == target.unsqueeze(dim=-1)).any(dim=-1).float()).item()
            total_top5 += torch.sum((pred_labels[:, :5] == target.unsqueeze(dim=-1)).any(dim=-1).float()).item()
            test_bar.set_description('Test Epoch: [{}/{}] Acc@1:{:.2f}% Acc@5:{:.2f}%'
                                     .format(epoch, epochs, total_top1 / total_num * 100, total_top5 / total_num * 100))

    return total_top1 / total_num * 100, total_top5 / total_num * 100



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
feature_dim, temperature, k = 128, 0.2, 200
batch_size, epochs = 1024, 100

# data prepare
train_data = utils.CIFAR10Pair(root='data', train=True, transform=utils.train_transform, download=True)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True,
                            drop_last=True)

memory_data = utils.CIFAR10Pair(root='data', train=True, transform=utils.test_transform, download=True)
memory_loader = DataLoader(memory_data, batch_size=batch_size, shuffle=False, num_workers=16, pin_memory=True)

test_data = utils.CIFAR10Pair(root='data', train=False, transform=utils.test_transform, download=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=16, pin_memory=True)

# model setup and optimizer config
model = Model()
# checkpoint = torch.load('results/128_1_5_128_100_model.pth')
# model.load_state_dict(checkpoint['model'],strict=False)
model.cuda()


flops, params = profile(model, inputs=(torch.randn(1, 3, 32, 32).cuda(),))
flops, params = clever_format([flops, params])
print('# Model Params: {} FLOPs: {}'.format(params, flops))


optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)

# WARMUP_STEPS = 15
# lr_scheduler = ConstantWithWarmup(optimizer, WARMUP_STEPS)

c = len(memory_data.classes)

# training loop
results = {'train_loss': [], 'test_acc@1': [], 'test_acc@5': []}
save_name_pre = '{}_{}_{}_{}_{}'.format(feature_dim, temperature, k, batch_size, epochs)
if not os.path.exists('results'):
    os.mkdir('results')
best_acc = 0.0

DECAY_EPOCHS = 40
DECAY = 0.1
current_learning_rate = 0.01
for param_group in optimizer.param_groups:
    param_group['lr'] = current_learning_rate

for epoch in range(1, epochs + 1):

    if epoch % DECAY_EPOCHS == 0 and epoch > 10:
        current_learning_rate = current_learning_rate * DECAY
        for param_group in optimizer.param_groups:
            param_group['lr'] = current_learning_rate
        print("Current learning rate has decayed to %f" %current_learning_rate)
    
    train_loss = train(model, train_loader, optimizer)
    results['train_loss'].append(train_loss)
    test_acc_1, test_acc_5 = test(model, memory_loader, test_loader)
    results['test_acc@1'].append(test_acc_1)
    results['test_acc@5'].append(test_acc_5)
    
    # lr_scheduler.step()

    # save statistics
    data_frame = pd.DataFrame(data=results, index=range(1, epoch + 1))
    data_frame.to_csv('results/{}_statistics.csv'.format(save_name_pre), index_label='epoch')
    if test_acc_1 > best_acc:
        print('updating best weights')
        best_acc = test_acc_1
        state = {
            'epoch': epoch,
            'state_dict': model.state_dict()}

        torch.save(state, 'results/{}_model.pth'.format(save_name_pre))


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
# Model Params: 11.26M FLOPs: 557.97M


Test Epoch: [1/100] Acc@1:36.07% Acc@5:86.22%: 100%|██████████| 10/10 [00:01<00:00,  5.44it/s]


updating best weights


Test Epoch: [2/100] Acc@1:38.91% Acc@5:87.58%: 100%|██████████| 10/10 [00:01<00:00,  5.46it/s]


updating best weights


...

Test Epoch: [94/100] Acc@1:77.09% Acc@5:98.81%: 100%|██████████| 10/10 [00:01<00:00,  5.36it/s]


updating best weights


Test Epoch: [95/100] Acc@1:77.20% Acc@5:98.79%: 100%|██████████| 10/10 [00:01<00:00,  5.31it/s]


updating best weights


Test Epoch: [100/100] Acc@1:77.04% Acc@5:98.79%: 100%|██████████| 10/10 [00:01<00:00,  5.52it/s]


In [30]:
import torchvision
model = Model().cuda()
label_ratio = 0.2

checkpoint = torch.load('results/128_0.2_200_1024_100_model.pth')
model.load_state_dict(checkpoint['state_dict'],strict=False)




optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.1)


train_data_classifier = torchvision.datasets.CIFAR10(root='data', train=True, transform=utils.test_transform, download=True)


batch_size = 128
train_data_classifier, _ = torch.utils.data.random_split(train_data_classifier,
                                               [label_ratio,1-label_ratio])
train_loader_classifier = DataLoader(train_data_classifier, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True,
                            drop_last=True)


test_data_classifier = torchvision.datasets.CIFAR10(root='data', train=False, transform=utils.test_transform, download=True)
_, test_data_classifier = torch.utils.data.random_split(test_data_classifier,
                                               [0.9,0.1])
test_loader_classifier = DataLoader(test_data_classifier, batch_size=1, shuffle=True, num_workers=16, pin_memory=True,
                            drop_last=True)




Files already downloaded and verified
Files already downloaded and verified


In [31]:
class SimpleNet(nn.Module):
    def __init__(self, input_dim=512, num_classes=10):
        super(SimpleNet, self).__init__()
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.dropout = nn.Dropout(0.85)
        self.fc1 = nn.Linear(self.input_dim, self.num_classes)
        

    def forward(self, x):
        x = self.fc1(x)
        x = self.dropout(x)
        x = F.relu(x)
        return x

In [32]:
def train_validate(model, loader, optimizer, is_train, epoch, use_cuda):

    loss_func = nn.CrossEntropyLoss()

    data_loader = loader
    # loader.train_loader if is_train else loader.test_loader

    model.train() if is_train else model.eval()

    desc = 'Train' if is_train else 'Validation'

    total_loss = 0
    total_acc = 0
    
    # tqdm_bar = tqdm(data_loader)
    for batch_idx, (x, y) in enumerate(data_loader):
        batch_loss = 0
        batch_acc = 0

        x = x.cuda() if use_cuda else x
        y = y.cuda() if use_cuda else y

        # Get features
        _,_, y_hat = model(x)

        loss = loss_func(y_hat, y)

        if is_train:
            # classifier_model.zero_grad()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


        # Reporting
        batch_loss = loss.item() / x.size(0)
        total_loss += loss.item()
        pred = y_hat.max(dim=1)[1]
        # print(y_hat.shape)
        # print(y)
        correct = pred.eq(y).sum().item()
        correct /= y.size(0)
        batch_acc = (correct * 100)
        total_acc += batch_acc
        
        # tqdm_bar.set_description('{} Epoch: [{}] Batch Loss: {:.4f} Batch Acc: {:.4f}'.format(desc, epoch, batch_loss, batch_acc))
    print('loss = '+str(total_loss))
    print('Epoch = '+str(epoch) + '; acc = ' + str(total_acc/(batch_idx + 1)))




In [33]:
for epoch in range(10):
    train_validate(model, train_loader_classifier, optimizer, True, epoch, True)
    print('========test =====')
    train_validate(model, test_loader_classifier, optimizer, False, 1, True)
    print('==================')
    print('==================')


loss = 100.68841075897217
Epoch = 0; acc = 63.88221153846154
========test =====
loss = 1064.1799957277253
Epoch = 1; acc = 69.0
loss = 74.34676796197891
Epoch = 1; acc = 72.45592948717949
========test =====
loss = 1096.6293103643402
Epoch = 1; acc = 68.7
loss = 73.68197029829025
Epoch = 2; acc = 72.08533653846153
========test =====
loss = 1133.0876705019327
Epoch = 1; acc = 67.8
loss = 73.6986711025238
Epoch = 3; acc = 71.15384615384616
========test =====
loss = 1160.5874557426432
Epoch = 1; acc = 63.3
loss = 74.30326420068741
Epoch = 4; acc = 70.01201923076923
========test =====
loss = 1190.3551252959296
Epoch = 1; acc = 60.9
loss = 75.63537466526031
Epoch = 5; acc = 68.80008012820512
========test =====
loss = 1268.412643907126
Epoch = 1; acc = 57.5
loss = 77.31826066970825
Epoch = 6; acc = 67.01722756410257
========test =====
loss = 1227.3328647884068
Epoch = 1; acc = 56.5
loss = 80.15788102149963
Epoch = 7; acc = 65.85536858974359
========test =====
loss = 1239.9973512906581
Epoch =